# House Prices: Advanced Regression Techniques (Kaggle)

## 05-adding-categorical-variables

Sources:
* Kaggle competition: https://www.kaggle.com/c/house-prices-advanced-regression-techniques
* Check missing values (Will Koehrsen): https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction by Will Koehrsen
* Neural net implementation (Yashu Seth): https://yashuseth.blog/2018/07/22/pytorch-neural-network-for-tabular-data-with-categorical-embeddings/ 
* Sklearn pipelines: https://medium.com/dunder-data/from-pandas-to-scikit-learn-a-new-exciting-workflow-e88e2271ef62
* Pipelines with dataframes (John Ramey): https://ramhiser.com/post/2018-04-16-building-scikit-learn-pipeline-with-pandas-dataframe/

## Problem description

**Previous**:

**kaggle-houseprice-01-linear-model-and-continuous-imputation.ipynb**
We try to predict house prices based on a number of continuous and categorical variables.
In the first step, the prediction will be made using only a small selection of continuous variables:

* LotFrontage: Linear feet of street connected to property
* LotArea: Lot size in square feet
* 1stFlrSF: First Floor square feet
* 2ndFlrSF: Second floor square feet
* TotalBsmtSF: Total square feet of basement area
* SalePrice: target variable

We will use a very simple network: a linear network with a single non-linearity.

**kaggle-houseprice-02-data-scaling.ipynb**

In order to make it a little easier for gradient descent to converge to a minimum, we will scale the input data to have 0 mean and a standard deviation of 1. For a discussion on why it is useful to scale input data, see https://stats.stackexchange.com/questions/249378/is-scaling-data-0-1-necessary-when-batch-normalization-is-used. We will not scale the target data, following this discussion: https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re.

**kaggle-houseprice-03-one-hot-for-missing-continuous.ipynb**

Instead of just replacing missing values in our dataset with the mean or the median of the respective column, we will now create a *one-hot encoded vector* to mark the previously *missing data* and add it to the data set. For the same reason that we used the *sklearn.preprocessing StandardScaler* we will now make use of the *sklearn.impute Imputer* to replace missing values. Also, to make this part of data processing a little easier to reuse, we will refactor the code into a function. 

* missing_LotFrontage: one-hot vector with 1 for each missing value in LotFrontage and 0 else

**kaggle-houseprice-04-pipeline-for-preprocessing.ipynb**

Instead of relying on self-written code for processing our continuous variables we will now delegate this part of the processing to sklearn transformers. Additionally, those transformers will be put in a pipeline so that the transformers don't have to be called individually every time. This will help keeping our code simple and clean, and produce consistent results for processing multiple data.

**Now:**

* Add categorical variables
* Extend pipeline to handle categoricals
* Create a function to pre-process an arbitrary amount of dataframes at once

We still need to add more data to our model. In contrast to the first set of continuous variables, this time we will add categorical variables. Categorical variables differ from continuous variables in the fact that there may or may not be a natural order to values of a categorical variable, and that we cannot use categorical variables to do meaningful calculations (e.g. to calculate the mean, or a sum). For more information see https://en.wikipedia.org/wiki/Level_of_measurement.
Often those variables are represented by strings. In order to let our network handle categorical variables, we need to convert them to numbers (also called *factors* or *codes*). Additionally, we will expand our pre-processing pipeline to also handle missing values for categorical variables. We will also create a function that let's us use the pipeline on an arbitrary amount of dataframes at the same time.

New variables:
* MSZoning: Identifies the general zoning classification of the sale.
* MSSubClass: Identifies the type of dwelling involved in the sale.

In [26]:
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder

In [27]:
# Show more rows and columns in the pandas output
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
#pd.set_option('display.width', 1000)

In [28]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Helpers

In [29]:
def show_missing(df, show_all=True):
    """    
    Shows absolute and relative number of missing values for each column of a dataframe,
    show_all=True also shows columns with no missing values.
    """
    mis_val_abs = df.isnull().sum()
    mis_val_rel = df.isnull().sum()/df.shape[0]
    mis_val_table = pd.concat([df.dtypes, mis_val_abs, mis_val_rel], axis=1)
    mis_val_table = mis_val_table.rename(columns={0: 'dtype', 1: 'Missing abs', 2: 'Missing rel'})

    if show_all:
        # Sort table descending by relative amount missing
        mis_val_table = mis_val_table.sort_values('Missing rel', ascending=False).round(3)
    else:
        # Sort table descending by relative amount missing, remove columns where no values are missing
        mis_val_table = mis_val_table[mis_val_table.iloc[:, 1] != 0].sort_values('Missing rel', ascending=False).round(3)
    
    return mis_val_table

## Load data

In [30]:
PATH = Path('../data/houseprice/')
#!dir {PATH}  # For Windows
!ls {PATH}

Der Befehl "ls" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [31]:
# Import training data
dep = ['SalePrice']
df_train = pd.read_csv(PATH/'train.csv', sep=',', header=0,
                       usecols=['MSZoning', 'MSSubClass', 'LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF',
                                'TotalBsmtSF', 'SalePrice'])
df_y = df_train[dep]
df_train = df_train.drop(dep, axis=1)
df_train.shape

(1460, 7)

In [32]:
# Import test data
df_test = pd.read_csv(PATH/'test.csv', sep=',', header=0,
                       usecols=['MSZoning', 'MSSubClass', 'LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF',
                                'TotalBsmtSF'])

In [33]:
# Define continuous and categorical columns
cat_names = ['MSZoning', 'MSSubClass']
cont_names = ['LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF', 'TotalBsmtSF']

## Pre-processing

First, we take a look at a couple of rows and some descriptive statistics. This gives us an idea about the scale of values, and helps to decide if some continuous variables should perhaps be treated as categorical. In this case all variables will be treated as continuous.

We also check for missing values. If we find any, we have two options: remove the rows that contain missing values (which might lead to losing a lot of observations), or replace them with other values so that the network can use them. Common values used as a replacement are the mean or the median of the series, or some constant.

In [34]:
df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,TotalBsmtSF,1stFlrSF,2ndFlrSF
0,60,RL,65.0,8450,856,856,854
1,20,RL,80.0,9600,1262,1262,0
2,60,RL,68.0,11250,920,920,866
3,70,RL,60.0,9550,756,961,756
4,60,RL,84.0,14260,1145,1145,1053


In [35]:
df_train[cont_names].describe()

,LotFrontage,LotArea,1stFlrSF,2ndFlrSF,TotalBsmtSF
count,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,70.049958,10516.828082,1162.626712,346.992466,1057.429452
std,24.284752,9981.264932,386.587738,436.528436,438.705324
min,21.000000,1300.000000,334.000000,0.000000,0.000000
25%,59.000000,7553.500000,882.000000,0.000000,795.750000
50%,69.000000,9478.500000,1087.000000,0.000000,991.500000
75%,80.000000,11601.500000,1391.250000,728.000000,1298.250000
max,313.000000,215245.000000,4692.000000,2065.000000,6110.000000


In [36]:
# Categorical variables can be of type int or string. To show all cat columns in describe,
# we need to convert them to the same dtype
df_train[cat_names].astype('category').describe()

,MSZoning,MSSubClass
count,1460,1460
unique,5,15
top,RL,20
freq,1151,536


In [37]:
pd.concat([show_missing(df_train), show_missing(df_test)], axis=1, sort=False)

,dtype,Missing abs,Missing rel,dtype,Missing abs,Missing rel
LotFrontage,float64,259,0.177,float64,227,0.156
MSSubClass,int64,0,0.000,int64,0,0.000
MSZoning,object,0,0.000,object,4,0.003
LotArea,int64,0,0.000,int64,0,0.000
TotalBsmtSF,int64,0,0.000,float64,1,0.001
1stFlrSF,int64,0,0.000,int64,0,0.000
2ndFlrSF,int64,0,0.000,int64,0,0.000


In [38]:
# The TypeSelector selects data from a dataframe based on its dtype. Credits to John Ramey, see sources on top.
class TypeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype=dtype
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=[self.dtype])        

In [39]:
def proc_df(cont_names, cat_names, *dataframes):
    """
    Pre-process arbitrary amount of dataframes with continuous and categorical variables.
    The respective fits are being calculated by combining all dataframes into a single
    dataframe.
    Returns one processed dataframe for each input dataframe.
    
    Parameters
    ----------
    cont_names : list
        List of column names for continuous variables.

    cat_names : list
        List of column names for categorical variables.

    *dataframes : pandas DataFrame(s)
        DataFrames to be processed.
    """
    
    df_combo = pd.DataFrame(columns=dataframes[0].columns)
    for arg in dataframes:
        df_combo = pd.concat([df_combo, arg], axis=0, sort=False)
        arg[cont_names] = arg[cont_names].astype('float64')
        arg[cat_names] = arg[cat_names].astype('category')
    
    # Convert columns in cont_names to *float64* dtype and the columns of cat_names to *category*.
    # This is necessary so that the TypeSelector in the pipeline can differentiate between cont and cat variables.
    # The pipeline can then apply different behaviour, according to the dtype.
    df_combo[cont_names] = df_combo[cont_names].astype('float64')
    df_combo[cat_names] = df_combo[cat_names].astype('category')
    
    # First, get names of columns with missing values.
    # The pipeline below then takes numeric features, in the order of appearance in the input dataframe.
    # The pipeline then takes categorical features in the order of appearance in the input dataframe.
    # All of these names are then merged to a list, and for the resulting dataframes.
    # This naming step is necessary because sklearn does not natively support pandas dataframes, and therefore
    #   all column names would be lost otherwise.
    missing_names = [f'mis_{name}' for name in df_combo.columns if df_combo[name].isnull().any()]
    ordered_cont_names = [col for col in df_combo.columns if col in cont_names]
    ordered_cat_names = [col for col in df_combo.columns if col in cat_names]
    names = missing_names + ordered_cont_names + ordered_cat_names
    
    preprocessing_pipeline = make_pipeline(
        FeatureUnion(transformer_list=[
            ('missing_features', make_pipeline(
                MissingIndicator(missing_values=np.nan)
            )),
            ('numeric_features', make_pipeline(
                TypeSelector('float64'),
                SimpleImputer(strategy='median'),
                StandardScaler()
            )),
            ('categorical_features', make_pipeline(
                TypeSelector('category'),
                SimpleImputer(strategy='most_frequent'),
                OrdinalEncoder()
            ))
        ])
    )
    preprocessing_pipeline.fit(df_combo)
        
    return (pd.DataFrame(preprocessing_pipeline.transform(arg), columns=names) for arg in dataframes)

In [40]:
df_train, df_test = proc_df(cont_names, cat_names, df_train, df_test)

In [41]:
df_train.head()

,mis_MSZoning,mis_LotFrontage,mis_TotalBsmtSF,LotFrontage,LotArea,TotalBsmtSF,1stFlrSF,2ndFlrSF,MSSubClass,MSZoning
0,0.0,0.0,0.0,-0.191815,-0.217879,-0.444278,-0.773861,1.207379,5.0,3.0
1,0.0,0.0,0.0,0.511940,-0.072044,0.477158,0.261075,-0.785025,0.0,3.0
2,0.0,0.0,0.0,-0.051064,0.137197,-0.299027,-0.610718,1.235375,5.0,3.0
3,0.0,0.0,0.0,-0.426400,-0.078385,-0.671232,-0.506205,0.978742,6.0,3.0
4,0.0,0.0,0.0,0.699608,0.518903,0.211621,-0.037170,1.671651,5.0,3.0


In [19]:
pd.concat([show_missing(df_train), show_missing(df_test)], axis=1, sort=False)

,dtype,Missing abs,Missing rel,dtype,Missing abs,Missing rel
mis_MSZoning,float64,0,0.0,float64,0,0.0
mis_LotFrontage,float64,0,0.0,float64,0,0.0
mis_TotalBsmtSF,float64,0,0.0,float64,0,0.0
LotFrontage,float64,0,0.0,float64,0,0.0
LotArea,float64,0,0.0,float64,0,0.0
TotalBsmtSF,float64,0,0.0,float64,0,0.0
1stFlrSF,float64,0,0.0,float64,0,0.0
2ndFlrSF,float64,0,0.0,float64,0,0.0
MSSubClass,float64,0,0.0,float64,0,0.0
MSZoning,float64,0,0.0,float64,0,0.0


In [20]:
df_train.describe()

,mis_MSZoning,mis_LotFrontage,mis_TotalBsmtSF,LotFrontage,LotArea,TotalBsmtSF,1stFlrSF,2ndFlrSF,MSSubClass,MSZoning
count,1460.0,1460.000000,1460.0,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,0.0,0.177397,0.0,0.028052,0.044221,0.012876,0.007762,0.024517,4.236986,3.028767
std,0.0,0.382135,0.0,1.033954,1.265751,0.995662,0.985452,1.018432,4.311548,0.632017
min,0.0,0.000000,0.0,-2.256162,-1.124590,-2.387009,-2.104493,-0.785025,0.000000,0.000000
25%,0.0,0.000000,0.0,-0.426400,-0.331566,-0.581018,-0.707584,-0.785025,0.000000,3.000000
50%,0.0,0.000000,0.0,-0.051064,-0.087452,-0.136754,-0.185018,-0.785025,4.000000,3.000000
75%,0.0,0.000000,0.0,0.465023,0.181772,0.559429,0.590546,0.913417,6.000000,3.000000
max,0.0,1.000000,0.0,11.443596,26.006354,11.479917,9.004498,4.032672,15.000000,4.000000


# PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
device

## Dataset, dataloader

In [ ]:
# Convert all data containers to tensors
t_train = torch.tensor(df_train.values, dtype=torch.float32, device=device)
t_y = torch.tensor(df_y.values, dtype=torch.float32, device=device)
#t_y = (t_y-t_y.mean())/t_y.std()

In [ ]:
# Dataset
train_ds = TensorDataset(t_train, t_y)

In [ ]:
# Dataloader
batch_size=64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

## Model

In [ ]:
class LinearNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layers
        self.linear1 = nn.Linear(6, 100)
        self.act1 = nn.ReLU()
        self.linear2 = nn.Linear(100, 1)        
    
    def forward(self, x):        
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        
        return x

In [ ]:
# Instantiate the model
model = LinearNet().to(device)

## Optimizer

In [ ]:
lr = 0.1
opt = torch.optim.Adam(model.parameters(), lr=lr)

## Loss

In [ ]:
loss_fn = F.mse_loss

## Train

In [ ]:
losses = []
def fit(num_epochs, model, loss_fn, opt):    
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            # Forward            
            preds = model(xb)
            loss = loss_fn(preds, yb)
            losses.append(loss)
            
            # Gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
            
        if epoch%20==0:
            print('Training loss:', loss_fn(model(t_train), t_y))

In [ ]:
# Train for 300 epochs
fit(num_epochs=300, model=model, loss_fn=loss_fn, opt=opt)

In [ ]:
plt.plot(losses)

In [ ]:
preds = model(t_train)

In [ ]:
torch.cat([preds, t_y.reshape(-1,1)], dim=1)[:10, :]

Even though we only added one additional column - the one-hot encoded vector for the missing values in *LotFrontage* - we could improve the training loss quite a bit

In [ ]:
plt.scatter(preds.detach().cpu().numpy(), t_y.reshape(-1,1).detach().cpu())